# Feature Engineering with SHAP values Experiment 1

SHAP Images of server with different number of poisoned clients
* number of malicious clients [0,1,2,3,4,5]

SHAP Images right after poisoning attack
* rounds [1,2,10,75,200]

Summation of Differences SHAP Images right after poisoning attack
* rounds [1,2,10,75,200]
* 5 times

## Google Colab

In [1]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
sys.path.append('/content/drive/My Drive/Colab Notebooks/federated_learning')

In [3]:
!pip install shap==0.40.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Experiments

In [4]:
from federated_learning.utils import SHAPUtil, experiment_util, Visualizer
from federated_learning import ClientPlane, Configuration, ObserverConfiguration
from federated_learning.server import Server
from datetime import datetime

## MNIST
(1) 5 → 4,
(2) 1 → 7,
(3) 3 → 8,

In [5]:
from federated_learning.nets import MNISTCNN
from federated_learning.dataset import MNISTDataset
import os
config = Configuration()
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = MNISTDataset
config.MODELNAME = config.MNIST_NAME
config.NETWORK = MNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.datasetObserverConfiguration = "MNIST"
neutral_label = 2

In [6]:
# Google Colab Settigns
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"

In [11]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)
visualizer = Visualizer(shap_util)

MNIST training data loaded.
MNIST test data loaded.
Create 200 clients with dataset of size 300


In [ ]:
import numpy as np
import copy
for i in range(199):
    experiment_util.set_rounds(client_plane, server, i+1)
    experiment_util.run_round(client_plane, server, i+1)
print("Run 199 finished")
old_params = copy.deepcopy(server.get_nn_parameters())

In [ ]:

#import torch
#torch.save(server.net.state_dict(), "/content/drive/My Drive/Colab Notebooks/temp/models/MNISTtrained2.model")

### alpha (5,4)

In [ ]:
config.FROM_LABEL = 5
config.TO_LABEL = 4

accuracies = []
recalls = []

server.net =  MNISTCNN()
server.net.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/temp/models/MNISTtrained2.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    client_plane.reset_default_client_nets()
    client_plane.reset_poisoning_attack()
    server.update_nn_parameters(old_params)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    file_name = './results/ex3/MNIST/5_4/_run_shap_values_{}_poisoned_clients_alpha_5_4.pdf'.format(j)
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracies.append(accuracy)
    recalls.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

In [ ]:
# Run 0
print(accuracies)
print(recalls)

### alpha (1,7)

In [ ]:
import copy
import torch
from federated_learning.nets import MNISTCNN
config.FROM_LABEL = 1
config.TO_LABEL = 7

server.net =  MNISTCNN()
server.net.load_state_dict(torch.load('temp/models/MNISTtrained2.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

In [ ]:
accuracies = []
recalls = []

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    client_plane.reset_default_client_nets()
    client_plane.reset_poisoning_attack()
    server.update_nn_parameters(old_params)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    file_name = './results/ex3/MNIST/1_7/4_run_shap_values_{}_poisoned_clients_alpha_1_7.pdf'.format(j)
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracies.append(accuracy)
    recalls.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

In [ ]:
# Run 0
print(accuracies)
print(recalls)

### alpha (3,8)

In [ ]:
import copy
import torch
from federated_learning.nets import MNISTCNN
config.FROM_LABEL = 3
config.TO_LABEL = 8

server.net =  MNISTCNN()
server.net.load_state_dict(torch.load('temp/models/MNISTtrained2.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

In [ ]:
accuracies = []
recalls = []

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    client_plane.reset_default_client_nets()
    client_plane.reset_poisoning_attack()
    server.update_nn_parameters(old_params)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    file_name = './results/ex3/MNIST/3_8/4_run_shap_values_{}_poisoned_clients_alpha_3_8.pdf'.format(j)
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracies.append(accuracy)
    recalls.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

In [ ]:
# Run 0
print(accuracies)
print(recalls)

## FashionMNIST
For Fashion-MNIST we experiment with 
(1) 5: sandal → 4: coat,
(2) 1: trouser → 3: dress, and 
(3) 8: Bag → 9: Ankle Boot.
['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker',  'Bag', 'Ankle Boot']


In [5]:
from federated_learning.nets import FMNISTCNN
from federated_learning.dataset import FMNISTDataset
import os
config = Configuration()
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = FMNISTDataset
config.MODELNAME = config.FMNIST_NAME
config.NETWORK = FMNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.datasetObserverConfiguration = "MNIST"
neutral_label = 2

In [6]:
# Google Colab Settigns
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"

In [7]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)
visualizer = Visualizer(shap_util)

FashionMnist training data loaded.
FashionMnist training data loaded.
Create 200 clients with dataset of size 300


In [ ]:
import numpy as np
import copy
for i in range(199):
    experiment_util.set_rounds(client_plane, server, i+1)
    experiment_util.run_round(client_plane, server, i+1)
print("Run 199 finished")
old_params = copy.deepcopy(server.get_nn_parameters())

In [ ]:
import torch
torch.save(server.net.state_dict(), "/content/drive/My Drive/Colab Notebooks/temp/models/FMNISTtrained.model")

### alpha (5,4)

In [8]:
from federated_learning.nets import FMNISTCNN
import torch
import copy
config.FROM_LABEL = 5
config.TO_LABEL = 4

server.net =  FMNISTCNN()
server.net.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/temp/models/FMNISTtrained.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

In [27]:
accuracies = []
recalls = []

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    client_plane.reset_default_client_nets()
    client_plane.reset_poisoning_attack()
    server.update_nn_parameters(old_params)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    file_name = '/content/drive/My Drive/Colab Notebooks/results/ex3/FMNIST/5_4/3_run_shap_values_{}_poisoned_clients_alpha_5_4.pdf'.format(j)
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracies.append(accuracy)
    recalls.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

In [ ]:

print(accuracies)
print(recalls)

## alpha(1,3)

In [47]:
from federated_learning.nets import FMNISTCNN
import torch
import copy
config.FROM_LABEL = 1
config.TO_LABEL = 3

server.net =  FMNISTCNN()
server.net.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/temp/models/FMNISTtrained.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

In [48]:
accuracies = []
recalls = []

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    client_plane.reset_default_client_nets()
    client_plane.reset_poisoning_attack()
    server.update_nn_parameters(old_params)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    file_name = '/content/drive/My Drive/Colab Notebooks/results/ex3/FMNIST/1_3/3_run_shap_values_{}_poisoned_clients_alpha_1_3.pdf'.format(j)
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracies.append(accuracy)
    recalls.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

In [ ]:
print(accuracies)
print(recalls)

alpha(8,9)

In [55]:
from federated_learning.nets import FMNISTCNN
import torch
import copy
config.FROM_LABEL = 8
config.TO_LABEL = 9

server.net =  FMNISTCNN()
server.net.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/temp/models/FMNISTtrained.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

In [68]:
accuracies = []
recalls = []

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    client_plane.reset_default_client_nets()
    client_plane.reset_poisoning_attack()
    server.update_nn_parameters(old_params)
    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    file_name = '/content/drive/My Drive/Colab Notebooks/results/ex3/FMNIST/8_9/4_run_shap_values_{}_poisoned_clients_alpha_8_9.pdf'.format(j)
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    print(clients)
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracies.append(accuracy)
    recalls.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    print(recall, precision, accuracy)
    print("Poisoned clients: {}".format(j))

In [ ]:
print(accuracies)
print(recalls)